<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')


from pymystem3 import Mystem

import imblearn
from imblearn.over_sampling import SMOTE


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [5]:
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [6]:
print(data['text'].head())

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: text, dtype: object


In [7]:
data['text'].head(3).values

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."],
      dtype=object)

- Комментарии на английском языке
- В данных нет пропусков.
- Присутствует дисбаланс классов.
- Присутствует спец. символы, даты и прочие цифры

**Очистку и лемматизацию мы будем применять ко всему дата-сету.** Мы может это сделать, поскольку наша функция просто будет изменять признак, без обучения.

In [8]:
lmtzr = WordNetLemmatizer()

def clear_text(text):
    text = text.lower()
    text = text.replace('\'', '')
    text = ' '.join(re.sub(r'[^a-z]', ' ', text).split()) 
    return text

def lemmatize(text):
    text = [lmtzr.lemmatize(word, 'v') for word in text.split()]
    return ' '.join(text)

In [9]:
lemmatize(clear_text(data['text'].values[0]))

'explanation why the edit make under my username hardcore metallica fan be revert they werent vandalisms just closure on some gas after i vote at new york dolls fac and please dont remove the template from the talk page since im retire now'

In [10]:
data['clear_text'] = data['text'].apply(clear_text) 

In [11]:
data['lemmatized'] = data['clear_text'].apply(lemmatize)

Избавимся от дисбаланса классов на обучающей выборке

In [12]:
data.head()

,text,toxic,clear_text,lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,daww he matches this background colour im seem...,daww he match this background colour im seemin...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really not trying to edit war its j...,hey man im really not try to edit war its just...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i cant make any real suggestions on impro...,more i cant make any real suggestions on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,you sir be my hero any chance you remember wha...


In [13]:
data['lemmatized'].values[7]

'your vandalism to the matt shirvington article have be revert please dont do it again or you will be ban'

In [14]:
data['clear_text'].values[7]

'your vandalism to the matt shirvington article has been reverted please dont do it again or you will be banned'

In [15]:
x_train, x_test, y_train, y_test = train_test_split(data['lemmatized'],
                                                    data['toxic'],
                                                    random_state=1,
                                                    test_size=.2,
                                                    stratify=data['toxic'])

Мы провели очистку текста, подготовили для создания "мешка". От идее использовать SMOTE для балансировки классов методом апсемплинга решено отказаться, поскольку время обучения и так достаточно большное

## Обучение

Мы будем использовть кросс-валидациюю, поэтому нельзя применять векторизацию сразу ко всему тренировочному дата-сету. Для этого используем пайплайн, чтобы на каждом фолде обучать tf-idf-vectorizer заново

In [18]:
stop_words = stopwords.words('english')

In [19]:
positive_weight = 1 / data['toxic'].value_counts(normalize=True)[1]
positive_weight

9.841344371679229

In [20]:
catboost_pipeline = Pipeline([
            ('tf-idf',  TfidfVectorizer(max_df=0.9, min_df=25,
                        max_features=7500,
                        stop_words=stop_words)),
            ('catboost', CatBoostClassifier(random_state=1,
                         n_estimators=1000,
                         verbose=200,
                         scale_pos_weight=positive_weight,
                         eval_metric='F1'))
])

params = {'catboost__max_depth': [4, 6],
          'catboost__learning_rate': [0.1, 0.2]}

In [21]:
%%time

grid = GridSearchCV(catboost_pipeline,
                    param_grid=params,
                    cv=3,
                    scoring='f1',
                    n_jobs=-1,
                    verbose=3)

grid.fit(x_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
0:	learn: 0.4860100	total: 851ms	remaining: 14m 10s
200:	learn: 0.8805521	total: 1m 58s	remaining: 7m 52s
400:	learn: 0.9100053	total: 3m 53s	remaining: 5m 48s
600:	learn: 0.9255997	total: 5m 46s	remaining: 3m 50s
800:	learn: 0.9369971	total: 7m 39s	remaining: 1m 54s
999:	learn: 0.9464519	total: 9m 30s	remaining: 0us
[CV 1/3] END catboost__learning_rate=0.1, catboost__max_depth=4;, score=0.730 total time= 9.7min
0:	learn: 0.7350429	total: 733ms	remaining: 12m 12s
200:	learn: 0.8778297	total: 2m	remaining: 7m 59s
400:	learn: 0.9082347	total: 3m 55s	remaining: 5m 51s
600:	learn: 0.9263125	total: 5m 49s	remaining: 3m 51s
800:	learn: 0.9390583	total: 7m 44s	remaining: 1m 55s
999:	learn: 0.9494889	total: 9m 36s	remaining: 0us
[CV 2/3] END catboost__learning_rate=0.1, catboost__max_depth=4;, score=0.729 total time= 9.8min
0:	learn: 0.7323163	total: 747ms	remaining: 12m 26s
200:	learn: 0.8800253	total: 2m 1s	remaining: 8m 2s
400:	lea

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tf-idf',
                                        TfidfVectorizer(max_df=0.9,
                                                        max_features=7500,
                                                        min_df=25,
                                                        stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('catboost',
                                        <catboost.core.CatBoostClassifier object at 0x7f8f0afa24f0>)]),
             n_jobs=-1,
             param_grid={'catboost__learning_rate': [0.1, 0.2],
                         'catboost__max_depth': [4, 6]},
             scoring='f1', verbose=3)

In [25]:
grid.best_score_

0.7535932903748134

Логистическая регрессия обучается побыстрее, так что можем поэкспериментировать с параметрами непосредственно векторизации

In [37]:
lr_pipeline = Pipeline([
            ('tf_idf',  TfidfVectorizer(stop_words=stop_words)),
            ('log_regr', LogisticRegression(random_state=1, max_iter=500))
])

params = {'tf_idf__max_df': [0.8, 0.9],
          'tf_idf__min_df': [25, 50, 75],
          'tf_idf__max_features': [8000, 12000]}

In [38]:
%%time

grid_lr = GridSearchCV(lr_pipeline,
                    param_grid=params,
                    cv=3,
                    scoring='f1',
                    n_jobs=-1,
                    verbose=3)

grid_lr.fit(x_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END tf_idf__max_df=0.8, tf_idf__max_features=8000, tf_idf__min_df=25;, score=0.733 total time=   6.7s
[CV 2/3] END tf_idf__max_df=0.8, tf_idf__max_features=8000, tf_idf__min_df=25;, score=0.736 total time=   6.4s
[CV 3/3] END tf_idf__max_df=0.8, tf_idf__max_features=8000, tf_idf__min_df=25;, score=0.730 total time=   6.2s
[CV 1/3] END tf_idf__max_df=0.8, tf_idf__max_features=8000, tf_idf__min_df=50;, score=0.728 total time=   6.2s
[CV 2/3] END tf_idf__max_df=0.8, tf_idf__max_features=8000, tf_idf__min_df=50;, score=0.733 total time=   6.3s
[CV 3/3] END tf_idf__max_df=0.8, tf_idf__max_features=8000, tf_idf__min_df=50;, score=0.725 total time=   6.1s
[CV 1/3] END tf_idf__max_df=0.8, tf_idf__max_features=8000, tf_idf__min_df=75;, score=0.725 total time=   6.3s
[CV 2/3] END tf_idf__max_df=0.8, tf_idf__max_features=8000, tf_idf__min_df=75;, score=0.727 total time=   6.3s
[CV 3/3] END tf_idf__max_df=0.8, tf_idf__max_featur

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tf_idf',
                                        TfidfVectorizer(stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('log_regr',
                                        LogisticRegression(max_iter=500,
                                                           random_state=1))]),
             n_jobs=-1,
             param_grid={'tf_idf__max_df': [0.8, 0.9],
                         'tf_idf__max_features': [8000, 12000],
                         'tf_idf__min_df': [25, 50, 75]},
             scoring='f1', verbose=3)

In [39]:
grid_lr.best_score_

0.7329460917137145

Модель Catboost на кросс-валидации показала лучшее качество, хотя и обучалась гораздо дольше. Проверим на тестовой выборке.

In [40]:
f1_score(y_test, grid.predict(x_test))

0.7500348140927446

## Выводы

Мы произвели очистку текста с его последующей лемматизацией.

Был создан мешок слов с помощью Tf-idf.

Был создан пайплайн для кросс-валидации и подбора гиперпараметров.

Требуемая метрика F1 в 0.75 была достигнута